In [1]:
%%capture
import numpy as np
import pandas as pd 
import requests
import time
import csv
import os
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
from multiprocessing import Pool
import json
import heapq
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from functools import reduce

In [2]:
%%capture
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.parse.earleychart import FilteredCompleteFundamentalRule
from nltk.stem import PorterStemmer
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
url="https://www.britannica.com/science/statistics"

In [ ]:
r = requests.get(url)
soup = bs(r.content, 'lxml')
soup

In [26]:
sub_links = soup.find_all('div', class_="toc ARTICLE mb-20")

In [30]:
sub_links

[<div class="toc ARTICLE mb-20">
 <div class="font-14 font-weight-bold pb-5 ml-15">Table of Contents</div>
 <ul class="list-unstyled mb-0" data-level="h1"><li data-target="#ref1"><a class="ml-25" href="/science/statistics">Introduction</a><div class="ml-40 toc-drawer sub-toc-drawer"></div></li><li data-target="#ref60691"><div class="d-flex align-items-center"><button aria-label="Toggle Heading" class="h1-link-drawer-button btn btn-xs btn-circle d-flex rounded" type="button"><em class="material-icons font-18" data-icon="keyboard_arrow_right"></em></button><a class="w-100" href="/science/statistics#ref60691">Descriptive statistics</a></div><div class="ml-40 toc-drawer sub-toc-drawer"><ul class="list-unstyled" data-level="h2"><li data-target="#ref60692"><a class="w-100" href="/science/statistics#ref60692">Tabular methods</a></li></ul><ul class="list-unstyled" data-level="h2"><li data-target="#ref60693"><a class="w-100" href="/science/statistics#ref60693">Graphical methods</a></li></ul><ul

In [25]:

all_text  = soup.find_all('section', id=True)

In [31]:
href_list = []

for div in sub_links:
    anchors = div.find_all('a', class_="w-100")
    for anchor in anchors:
        href = anchor['href']
        href_list.append(href)

print(href_list)

['/science/statistics#ref60691', '/science/statistics#ref60692', '/science/statistics#ref60693', '/science/statistics/Numerical-measures', '/science/statistics/Numerical-measures#ref60695', '/science/statistics/Numerical-measures#ref60696', '/science/statistics/Probability', '/science/statistics/Probability#ref60698', '/science/statistics/Random-variables-and-probability-distributions', '/science/statistics/Random-variables-and-probability-distributions#ref60700', '/science/statistics/Random-variables-and-probability-distributions#ref60701', '/science/statistics/Random-variables-and-probability-distributions#ref60702', '/science/statistics/Random-variables-and-probability-distributions#ref60703', '/science/statistics/Estimation', '/science/statistics/Estimation#ref60705', '/science/statistics/Estimation-of-a-population-mean', '/science/statistics/Estimation-of-a-population-mean#ref60707', '/science/statistics/Estimation-of-a-population-mean#ref60708', '/science/statistics/Hypothesis-te

In [33]:
new_href_list = []

for href in href_list:
    clean_href = href.replace("/science/statistics/", "").split("#")[0]
    new_href_list.append(clean_href)

print(new_href_list)

['/science/statistics', '/science/statistics', '/science/statistics', 'Numerical-measures', 'Numerical-measures', 'Numerical-measures', 'Probability', 'Probability', 'Random-variables-and-probability-distributions', 'Random-variables-and-probability-distributions', 'Random-variables-and-probability-distributions', 'Random-variables-and-probability-distributions', 'Random-variables-and-probability-distributions', 'Estimation', 'Estimation', 'Estimation-of-a-population-mean', 'Estimation-of-a-population-mean', 'Estimation-of-a-population-mean', 'Hypothesis-testing', 'Hypothesis-testing', 'Experimental-design', 'Experimental-design', 'Experimental-design', 'Experimental-design', 'Experimental-design', 'Experimental-design', 'Experimental-design', 'Residual-analysis', 'Residual-analysis', 'Residual-analysis', 'Residual-analysis', 'Residual-analysis', 'Residual-analysis', 'Residual-analysis', 'Residual-analysis', 'Sample-survey-methods', 'Sample-survey-methods']


In [34]:
unique_href_list = list(set(new_href_list))
print(unique_href_list)

['Sample-survey-methods', 'Estimation', 'Experimental-design', 'Residual-analysis', 'Probability', 'Hypothesis-testing', 'Estimation-of-a-population-mean', 'Random-variables-and-probability-distributions', '/science/statistics', 'Numerical-measures']


In [52]:
my_list = [item for item in unique_href_list if item != '/science/statistics']

# Aggiungi una barra all'inizio di ogni elemento
sub_pages_final = ['https://www.britannica.com/science/statistics/' + item for item in my_list]

In [53]:

sub_pages_final= list(reversed(sub_pages_final))
sub_pages_final.insert(0,'https://www.britannica.com/science/statistics')
print(sub_pages_final)

['https://www.britannica.com/science/statistics', 'https://www.britannica.com/science/statistics/Numerical-measures', 'https://www.britannica.com/science/statistics/Random-variables-and-probability-distributions', 'https://www.britannica.com/science/statistics/Estimation-of-a-population-mean', 'https://www.britannica.com/science/statistics/Hypothesis-testing', 'https://www.britannica.com/science/statistics/Probability', 'https://www.britannica.com/science/statistics/Residual-analysis', 'https://www.britannica.com/science/statistics/Experimental-design', 'https://www.britannica.com/science/statistics/Estimation', 'https://www.britannica.com/science/statistics/Sample-survey-methods']


In [ ]:
list_text=[]
for link in sub_pages_final:
  r2 = requests.get(link)
  soup2 = bs(r2.content, 'lxml')
  all_text2  = soup2.find_all('section', id=True)
  for section in all_text2:
    section_id = section['id']
    print(section)
    text = section.get_text()
    text = text.split('Get a Britannica Premium subscription')[0]
    text = text.split('Britannica Money')[0]
    text = text.split('More From Britannica probability and statistics: The rise of statistics')[0]
    list_text.append(text)
    #print(f"Testo: {text.strip()}")
    print("------------")
  

In [71]:
def salva_lista_su_file(lista, nome_file):
    with open(nome_file, 'w') as file:
        for elemento in lista:
            file.write(elemento + '\n')

In [79]:
def rimuovi_duplicati(lista):
    visto = set()
    nuova_lista = []
    for elemento in lista:
        if elemento not in visto:
            visto.add(elemento)
            nuova_lista.append(elemento)
    return nuova_lista

In [80]:
len(list_text)

43

In [81]:
lista_senza_duplicati = rimuovi_duplicati(list_text)

In [105]:
len(lista_senza_duplicati)


43

In [72]:
salva_lista_su_file(list_text, "lista_text.txt")

In [127]:
Question_list=["What is Statistics?", "What is Descriptive Statistics?", "What are tabular methods?", "What are Graphical methods?", " What are Numerical measures? ", " Teach me about Outliers", "Can you explain the Exploratory data analysis?", "tell me about Random variables and probability distributions", "what are Special probability distributions?","Can you explain the binomial distribution?", "Can you explain the Poisson distribution?", "Can you explain the Normal distribution?","describe the Estimation of a population mean", "describe the Estimation of other parameters", "tell me the Estimation procedures for two populations", "what is Hypothesis testing?", "what are Bayesian methods?", "what is Probability? ", "tell me about the Events and their probabilities", "Explain Residual analysis", "what is Model building In regression analysis?", "what is Correlation?", "explain Time series and forecasting", "what are the Nonparametric methods?", "explain the Statistical quality control", " what is Acceptance sampling?", "tell me about Statistical process control", "what is an Experimental design in statistics?", "what is Analysis of variance and significance testing?", "what is Regression and correlation analysis?", "decsribe a Regression model", "can you explain the Least squares method?", "tell me about: Analysis of variance and goodness of fit", "what is Significance testing In a regression study?", " what is estimation in statistics and probability?", "what is Sampling and sampling distributions in statistics?", "what are Sample survey methods?", "what is Decision analysis in statistics?"  ]

In [128]:
salva_lista_su_file(Question_list, "Question_list_britannica.txt")